# Developing an analytical platform for evaluating the role of forest biorefineries in achieving a sustainable bioeconomy

# Overview:
--------------
This script performs a comprehensive Techno-Economic Analysis (TEA) for bioethanol, vanillin, and furfural production in a biorefinery setting. It evaluates 100 variations of vanillin and furfural production by varying the percentage of woodchips allocated to vanillin and furfural synthesis from 100% down to 1%. The remaining woodchips are utilized for steam production, and any residual biomass is converted into ash, which incurs associated costs.


# Key Features:
--------------
- **Capital Expenditure (CAPEX) Calculation:** Computes total CAPEX by summing detailed infrastructure costs.
- **Revenue Generation from Products:** Treats both vanillin and steam as revenue-generating products.
- **Financial Metrics:** Calculates essential financial indicators including Net Present Value (NPV), Internal Rate of Return (IRR), and Payback Period for each production variation.
- **Error Handling:** Incorporates robust error handling to manage and report issues for individual production scenarios without halting the entire analysis.
- **Output Generation:** Outputs the TEA results for all variations into an Excel file for easy analysis and visualization.


# Requirements:
--------------
The following Python libraries and tools are essential for running the TEA script:
1. **`pandas`**: Used for data manipulation and analysis. Install with: `pip install pandas == 1.3.3`
2. **`numpy`**: Used for numerical operations and handling arrays. Install with: `pip install numpy == 1.21.2`
3. **`openpyxl`**: Used for writing Excel files. Install with: `pip install openpyxl == 3.0.8`

## Techno-economic analysis

In [ ]:
import pandas as pd
import numpy as np
import numpy_financial as npf 

In [ ]:
# Load the Excel file
file_path_excel = "./input/Vanillin.xlsx" # Set the path to the Vanillin.xlsx input file (assumed to be in the local 'input' folder)
excel_data = pd.ExcelFile(file_path_excel)

In [ ]:
# Production parameters
production_days_per_year = 364
operating_hours_per_day = 22 / 3  # Assuming Vanillin is 1/3 of production
total_operating_hours_per_year = production_days_per_year * operating_hours_per_day

In [ ]:
# Function to read and extract specific rows and columns for each sheet
def extract_inventory_data(sheet_name):
    # Read rows 17 to 34 and columns A, B, C (which are indexed as 0, 1, 2)
    df = pd.read_excel(excel_data, sheet_name=sheet_name, skiprows=16, usecols="A:C", nrows=18)
    
    # Rename the columns for clarity
    df.columns = ['name', 'amount', 'unit']
    
    # Drop rows with missing values (if necessary)
    df_cleaned = df.dropna(subset=['name', 'amount', 'unit'])
    
    return df_cleaned

In [ ]:
# Function to calculate the total inventory per year, scaling based on woodchips usage
def calculate_yearly_inventory(inventory_data, operating_hours_per_year):
    # Find the base amount of woodchips from the input data
    base_woodchips_amount = inventory_data[inventory_data['name'] == 'wood chips production, softwood, at sawmill']['amount'].values[0]
    
    # Calculate the yearly amount of woodchips (this is correct as per your current code)
    woodchips_per_year = base_woodchips_amount * 4 * operating_hours_per_day * production_days_per_year
    
    # Scale all other items based on the ratio of woodchips_per_year to base_woodchips_amount
    scaling_factor = woodchips_per_year / base_woodchips_amount
    
    # Apply the scaling factor to all items
    inventory_data['amount_per_year'] = inventory_data['amount'] * scaling_factor
    
    return inventory_data

In [ ]:
# Function to process all sheets and calculate yearly inventory
def process_all_sheets():
    results = {}
    
    # Loop through each sheet in the Excel file
    for sheet_name in excel_data.sheet_names:
        if sheet_name == "skip this sheet":
            continue  # Skip this sheet
        
        # Extract inventory data for the current sheet
        inventory_data = extract_inventory_data(sheet_name)
        
        # Calculate total inventory per year
        yearly_inventory = calculate_yearly_inventory(inventory_data, total_operating_hours_per_year)
        
        # Store the results for this sheet
        results[sheet_name] = yearly_inventory
    
    return results


In [ ]:
# Process all the sheets and extract inventory data
all_inventory_data = process_all_sheets()

# Display the yearly inventory for each sheet
for sheet_name, inventory_data in all_inventory_data.items():
    print(f"Yearly Inventory for {sheet_name}:")
    print(inventory_data[['name', 'amount_per_year', 'unit']])
    print("\n")

In [ ]:
# Prices for each input/output, value ($/unit)
prices = {
    "market for electricity, medium voltage": 0.13,
    "market for heat, from steam, in chemical industry": 0.01386,
    "market for sodium hydroxide, without water, in 50% solution state": 0.098,
    "wood chips production, softwood, at sawmill": 0.145,
    "market for acetic acid, without water, in 98% solution state": 44,
    "market for carbon dioxide, liquid": 0.004,
    "market for dichloromethane": 47.93,
    "water production, deionised": 1.27,
    "Vanillin production": 225,
    "market for wastewater, average": 0.45,
    "market for municipal solid waste": 0.03726,
    "market for wood ash mixture, pure": 0.103
}

### Financial parameters

In [ ]:
# Financial parameters
financial_params = {
    "min_IRR": 0.20,
    "discount_rate": 0.10,
    "interest_rate": 0.06,
    "debt_percent": 1.00,
    "depreciation_period": 20,
    "loan_period": 20,
    "amortization_period": 20,
    "tax_rate": 0.20,
    "project_life_time": 20
}

In [ ]:
# Installed costs 
installed_costs = {
    "Pretreatment": 6535,
    "Filterationforall": 1506,
    "Saccharification": 1638,
    "Fermentationforbioethanol": 5774,
    "Reactorfurfural": 3587,
    "Wastewatertreatmentforall": 6203,
    "Lignin extraction (Alkaline oxidation)": 1239,
    "Waste lignin combustion": 11704,
    "Steam generation": 469
}

# Equipment costs 
equipment_costs = {
    "Pretreatment": 5357,
    "Filterationforall": 837,
    "Saccharification": 1342,
    "Fermentationforbioethanol": 4734,
    "Reactorfurfural": 2937,
    "Wastewatertreatmentforall": 4211,
    "Lignin extraction (Alkaline oxidation)": 583,
    "Waste lignin combustion": 6450,
    "Steam generation": 160
}

# Installation costs 
installation_costs = {
    "Pretreatment": 1178,
    "Filterationforall": 669,
    "Saccharification": 296,
    "Fermentationforbioethanol": 1041,
    "Reactorfurfural": 650,
    "Wastewatertreatmentforall": 1993,
    "Lignin extraction (Alkaline oxidation)": 656,
    "Waste lignin combustion": 5254,
    "Steam generation": 310
    
}

#Total equipment costs
total_equipment_costs = sum(equipment_costs.values())   # $
    
#Total installation costs
total_installation_costs = sum(installation_costs.values())     # $   
    
# Capital costs
capital_costs = total_equipment_costs + total_installation_costs  # $


# Other costs
other_costs = {
    "Insurance": 6666.67,  # $
    "Misc": 3333.33        # $
}

# Labor costs
labor_costs = {
    "Labor_hour_pay_rate": 15,       # $
    "Payroll_cost_percentage": 0.11  # 11%
}

In [ ]:
# Operation parameters
production_days_per_year = 364
operating_hours_per_day = 22 / 3  # Divided by 3 products
operation_params = {
    "total_operating_hours_per_year": production_days_per_year * operating_hours_per_day,  # total operating hours per year
    "total_labor_hours": production_days_per_year * (24/3)     # total labor hours
}

In [ ]:
print(operation_params)

In [ ]:
# Woodchips and steam data
woodchips_total = 0.306  # kg
woodchips_per_hour = 4  # kg/h
calorific_value = 27.15  # MJ/kg for woodchips
efficiency = 0.80  # Conversion efficiency to steam
ash_percentage = 0.08  # 8% ash from woodchips used for steam

### TEA Calculation

In [ ]:
# Function to calculate total cost and revenue
def calculate_cost_and_revenue(inventory_data, prices):
    total_cost = 0
    total_revenue = 0
    
    # Loop through the inventory data
    for index, row in inventory_data.iterrows():
        name = row['name']
        yearly_amount = row['amount_per_year']  # Use yearly amount instead of raw amount
                
        # Get the price for the item from the prices dictionary
        unit_price = prices.get(name, 0)
        
        # Handle Vanillin production revenue (always treated as revenue)
        if "Vanillin" in name:
            unit_price = prices.get("Vanillin production", 225)
            total_revenue += yearly_amount * unit_price
        
        # Handle "market for heat, from steam, in chemical industry" revenue only if amount is negative
        elif name == "market for heat, from steam, in chemical industry":
            if yearly_amount < 0:
                # Treat as revenue (convert negative amount to positive revenue)
                total_revenue += abs(yearly_amount) * unit_price
            else:
                # Treat as cost
                total_cost += yearly_amount * unit_price
                
          # For all other items, determine if it's a cost or revenue based on amount's sign
        else:
            if yearly_amount >= 0:
                # Positive amount: input (cost)
                total_cost += yearly_amount * unit_price
            else:
                # Negative amount: output (revenue)
                total_revenue += abs(yearly_amount) * unit_price
    
    return total_cost, total_revenue      
                

In [ ]:
# Function to process all sheets and calculate cost and revenue for each variation
def process_all_sheets_and_calculate_cost_revenue():
    results = {}

    for sheet_name in excel_data.sheet_names:
        if sheet_name == "skip this sheet":
            continue  # Skip this sheet
        
        # Extract inventory data for the current sheet
        inventory_data = extract_inventory_data(sheet_name)
        
        # Calculate total inventory per year
        yearly_inventory = calculate_yearly_inventory(inventory_data, total_operating_hours_per_year)
        
        # Calculate total cost and revenue for this sheet (variation)
        total_cost, total_revenue = calculate_cost_and_revenue(yearly_inventory, prices)
        
        # Store the results (cost and revenue) for this sheet
        results[sheet_name] = {
            "total cost": total_cost,
            "total revenue": total_revenue
        }
    
    return results

In [ ]:
# Run the process for all sheets and get cost and revenue for each variation
all_cost_revenue_results = process_all_sheets_and_calculate_cost_revenue()

In [ ]:
# Display the calculated cost and revenue for each variation
for sheet_name, result in all_cost_revenue_results.items():
    print(f"Results for {sheet_name}:")
    print(f"  total cost: {result['total cost']}")
    print(f"  total revenue: {result['total revenue']}")
    print("\n")

In [ ]:
# Function to calculate yearly cash flow, NPV, IRR, and Payback Period
def calculate_financial_metrics(total_revenue, total_cost, capital_costs, financial_params, total_equipment_costs, total_installation_costs):
    depreciation = total_equipment_costs / financial_params["depreciation_period"]
    amortisation = total_installation_costs / financial_params["amortization_period"]
    interest = capital_costs * financial_params["interest_rate"] * financial_params["debt_percent"]
    total_deprecation_amortisation_interest = depreciation + amortisation + interest
    
    #EBITDA
    EBITDA = total_revenue - total_cost
    
    
    # Store results for each year
    net_income_before_tax_years = []
    net_income_after_tax_years = []
    net_cash_flow_years = []
    pv_of_cash_flows_years = []
    total_fixed_costs_years = []
    
    
    # Loop over years from 1 to 20
    for year in range(1, 21):
        if year == 1:
            # Year 1 calculations
            net_income_before_tax = EBITDA - total_deprecation_amortisation_interest
        else:
            # Years 2-20 calculations: Adjust the interest, depreciation, and amortisation
            net_income_before_tax = EBITDA - (financial_params["interest_rate"] * (capital_costs * financial_params["debt_percent"] - (depreciation + amortisation) * (year - 1))) - depreciation + amortisation
        
        
        # Calculate Net Income After Tax with conditional logic
        if net_income_before_tax > 0:
            net_income_after_tax = net_income_before_tax * (1 - financial_params["tax_rate"])
        else:
            net_income_after_tax = net_income_before_tax  # No tax applied if Net Income Before Tax is negative
        
        
        
        # Net Cash Flow (after tax)
        net_cash_flow = net_income_after_tax + depreciation + amortisation
        
        # Present Value (PV) of Cash Flows (after tax)
        pv_of_cash_flow = net_cash_flow / ((1 + financial_params["discount_rate"]) ** year)
            
        # Append results for this year
        net_income_before_tax_years.append(net_income_before_tax)
        net_income_after_tax_years.append(net_income_after_tax)
        net_cash_flow_years.append(net_cash_flow)
        pv_of_cash_flows_years.append(pv_of_cash_flow)
    
    # Calculate NPV (Net Present Value) from the net cash flows
    npv = npf.npv(financial_params["discount_rate"], [-capital_costs] + net_cash_flow_years)

    # Calculate IRR (Internal Rate of Return)
    irr = npf.irr([-capital_costs] + net_cash_flow_years)
    

    # Calculate Payback Period
    cumulative_cash_flow = -capital_costs
    payback_period = None
    for year, cash_flow in enumerate(net_cash_flow_years, start=1):
        cumulative_cash_flow += cash_flow
        if cumulative_cash_flow >= 0:
            payback_period = year
            break

    if payback_period is None:
        payback_period = -1  # No payback period if cash flow never covers capital costs

    # Return all results in a dictionary
    return {
        "EBITDA": EBITDA,
        "net_income_before_tax": net_income_before_tax_years,
        "net_income_after_tax": net_income_after_tax_years,
        "net_cash_flow": net_cash_flow_years,
        "pv_of_cash_flows": pv_of_cash_flows_years,
        "NPV": npv,
        "IRR": irr,
        "Payback Period": payback_period
    }
  
    

In [ ]:
# Function to process all sheets and calculate total cost, revenue, and financial metrics
def process_all_sheets():
    results = {}

    for sheet_name, inventory_data in all_inventory_data.items():
        # Step 1: Calculate total cost and revenue for the current sheet
        total_cost, total_revenue = calculate_cost_and_revenue(inventory_data, prices)
        
        # Step 2: Calculate financial metrics (NPV, IRR, Payback Period, etc.)
        financial_metrics = calculate_financial_metrics(
            total_revenue=total_revenue,
            total_cost=total_cost,
            capital_costs=capital_costs,
            financial_params=financial_params,
            total_equipment_costs=total_equipment_costs,
            total_installation_costs=total_installation_costs
        )
        
        # Store the results for this sheet
        results[sheet_name] = {
            "EBITDA": financial_metrics["EBITDA"],
            "NPV": financial_metrics["NPV"],
            "IRR": financial_metrics["IRR"],
            "Payback Period": financial_metrics["Payback Period"],
            "Net Income Before Tax": financial_metrics["net_income_before_tax"],
            "Net Income After Tax": financial_metrics["net_income_after_tax"],
            "Net Cash Flow": financial_metrics["net_cash_flow"],
            "PV of Cash Flows": financial_metrics["pv_of_cash_flows"]
        }

    return results

In [ ]:
# Function to process all sheets and calculate total cost, revenue, and financial metrics
def process_all_sheets_and_calculate_metrics():
    results = {}

    for sheet_name in excel_data.sheet_names:
        if sheet_name == "skip this sheet":
            continue  # Skip this sheet if necessary
        
        # Extract inventory data for the current sheet
        inventory_data = extract_inventory_data(sheet_name)
        
        # Calculate total inventory per year
        yearly_inventory = calculate_yearly_inventory(inventory_data, total_operating_hours_per_year)
        
        # Step 1: Calculate total cost and revenue for the current sheet
        total_cost, total_revenue = calculate_cost_and_revenue(yearly_inventory, prices)
        
        # Step 2: Calculate financial metrics (NPV, IRR, Payback Period, etc.)
        financial_metrics = calculate_financial_metrics(
            total_revenue=total_revenue,
            total_cost=total_cost,
            capital_costs=capital_costs,
            financial_params=financial_params,
            total_equipment_costs=total_equipment_costs,
            total_installation_costs=total_installation_costs
        )
        
        # Store the results for this sheet
        results[sheet_name] = {
            "EBITDA": financial_metrics["EBITDA"],
            "NPV": financial_metrics["NPV"],
            "IRR": financial_metrics["IRR"],
            "Payback Period": financial_metrics["Payback Period"],
            "Net Income Before Tax": financial_metrics["net_income_before_tax"],
            "Net Income After Tax": financial_metrics["net_income_after_tax"],
            "Net Cash Flow": financial_metrics["net_cash_flow"],
            "PV of Cash Flows": financial_metrics["pv_of_cash_flows"]
        }

    return results

In [ ]:
# Process all sheets and calculate financial metrics
all_financial_results = process_all_sheets_and_calculate_metrics()

In [ ]:
# Display the calculated results for each sheet
for sheet_name, result in all_financial_results.items():
    print(f"Results for {sheet_name}:")
    print(f"  EBITDA: {result['EBITDA']}")
    print(f"  NPV: {result['NPV']}")
    print(f"  IRR: {result['IRR']}")
    print(f"  Payback Period: {result['Payback Period']} years")
    print(f"  Net Income Before Tax: {result['Net Income Before Tax']}")
    print(f"  Net Income After Tax: {result['Net Income After Tax']}")
    print(f"  Net Cash Flow: {result['Net Cash Flow']}")
    print(f"  PV of Cash Flows: {result['PV of Cash Flows']}")
    print("\n")

In [ ]:
# Example of saving the results to an Excel file (optional)
summary_df = pd.DataFrame.from_dict(all_financial_results, orient='index')
output_file_path = "./output/Financial_Results_Vanillin.xlsx" # Set the output file path for saving financial results to a local 'output' folder

summary_df.to_excel(output_file_path)

In [ ]:
print(f"Financial results saved to {output_file_path}")

In [ ]:
# Initialize an empty list to store Vanillin production data
vanillin_production_data = []

# Process all the sheets and extract Vanillin production for each variation
for sheet_name, inventory_data in all_inventory_data.items():
    # Filter the row where 'name' contains 'Vanillin'
    vanillin_row = inventory_data[inventory_data['name'].str.contains("Vanillin")]
    
    if not vanillin_row.empty:
        vanillin_name = vanillin_row['name'].values[0]  # Get the name (e.g., "Vanillin 100% production")
        vanillin_amount = vanillin_row['amount_per_year'].values[0]  # Get the amount
        
        # Append the data to the list
        vanillin_production_data.append({
            'Vanillin Variation': vanillin_name,
            'Amount': vanillin_amount
        })

# Convert the list to a DataFrame
vanillin_df = pd.DataFrame(vanillin_production_data)

# Specify the output file path
output_file_path = "./output/Vanillin_Production_Data.xlsx" # Set the output file path to save results in a local 'output' folder

# Save the DataFrame to an Excel file
vanillin_df.to_excel(output_file_path, index=False)

print(f"Vanillin production data saved to {output_file_path}")
